# Predicting Mortality Rate Among Non-traumatic Intracerebral Haemorrhage Patients in Intensive Care Unit (ICU) Using MIMIC-IV Dataset

In this project, we will investigate the mortality rate of patients with non-traumatic intracerebral haemorrhage in the intensive care unit. We intend to compare the accuracy of mortality rate using three different classifiers, namely logistic regression, k-nearest neighbour, and random forest. 

Why is it critical to determine the mortality rate of patients with non-traumatic intracerebral haemorrhage? Stroke is a major health issue that affects people all over the world. Nontraumatic intracerebral haemorrhage (ICH) is a type of stroke in which a hematoma forms within the brain parenchyma, with or without blood extension into the ventricles (Rajashekar & Liang 2022). ICH affects over 2 million people each year and accounts for up to 10%–15% of all strokes (Cordonnier et al., 2018; Vilela & Wiesmann, 2020), with long-term morbidity and a high mortality rate when compared to other types of strokes (Tatlisumak et al., 2018). Carolei et al. (1997) and Van Asch et al. (2010) estimate that approximately 35% of ICH patients die within seven days, and approximately 50% die within 30 days. Furthermore, the global cost of caring for ICH patients, particularly those in intensive care units, is enormous (ICUs). As a result, it becomes critical to predict the mortality rate among ICH patients as early as possible.

ICH scores were previously used to determine mortality (Hemphill et al., 2001). The ICH score is calculated in the first minute of a patient's visit using the Glasgow Coma Scale (GCS), haemorrhage volume (cm3), intraventricular haemorrhage status, haemorrhage site (supra or infratentorial), and patient's age (Rahmani et al., 2018). The ICH score must be assessed by experienced radiologists and neurologists, which is expensive, time-consuming, and difficult for inexperienced users (Nie et al., 2021). APACHE II and SAPS II are reliable predictors of mortality in ICU patients (Lee et al., 2015; Godinjak et al., 2016). These scores use traditional statistical analysis to identify the most relevant covariates from preselected features, resulting in model oversimplification and discretization, reduced covariates, and poor model performance (Nie et al., 2021).

As a result of the limitations of the methods mentioned above, it is necessary to employ machine learning algorithms. The advantage of machine learning methods is that they allow for the automatic discovery of important variables and empirical patterns in data. It starts with the patient's observation and selects many variables to identify the combination that can predict the outcomes reliably. Machine learning can deal with variables with complex interactions using a variety of algorithms that do not make linear assumptions. Another advantage of machine learning is that it can process many predicted values, allowing for a more comprehensive and in-depth exploration of big data.


# About MIMIC dataset


MIMIC is a very large dataset, hence we only selected some part of it. We included the tables from four modules: core, ed, hosp, and icu:
* core:**The core module stores patient tracking information necessary for any data analysis using MIMIC-IV. The core module contains three tables: patients, admissions, and transfers. These tables provide demographics for the patient, a record for each hospitalization, and a record for each ward stay within a hospitalization.**
* ed:**Patient stays are tracked in the edstays table. Each row of the edstays table has a unique stay_id, which represents a unique patient stay in the ED. The edstays table contains the following columns: subject_id, hadm_id, stay_id, intime, and outtime. The intime indicates the time at which the patient was admitted to the ED, and the outtime indicates the time at which the patient was discharged from the ED. If the patient was admitted to the hospital following their ED stay, the hadm_id column will be populated with an identifier representing their hospital stay. hadm_id can be linked with the hadm_id in MIMIC-IV to obtain further detail about the patient’s hospital stay. Finally, each individual is assigned a unique subject_id, and patients with multiple ED stays will have the same subject_id across stays in the edstays table. Note that subject_id can be linked with MIMIC-IV to obtain patient demographics. subject_id can also be linked with the PatientID DICOM attribute in MIMIC-CXR to obtain chest x-rays for patients if they were taken**
* hosp:**The hosp module contains data derived from the hospital wide EHR. These measurements are predominantly recorded during the hospital stay, though some tables include data from outside the hospital as well (e.g. outpatient laboratory tests in labevents). Information includes laboratory measurements (labevents, d_labitems), microbiology cultures (microbiologyevents, d_micro), provider orders (poe, poe_detail), medication administration (emar, emar_detail), medication prescription (prescriptions, pharmacy), hospital billing information**
* icu:**The icu module contains data sourced from the clinical information system at the BIDMC: MetaVision (iMDSoft). MetaVision tables were denormalized to create a star schema where the icustays and d_items tables link to a set of data tables all suffixed with events. Data documented in the icu module includes intravenous and fluid inputs (inputevents), patient outputs (outputevents), procedures (procedureevents), information documented as a date or time (datetimeevents), and other charted information (chartevents). All events tables contain a stay_id column allowing identification of the associated ICU patient in icustays, and an itemid column allowing identification of the concept documented in d_items.**

We will take a glance at these modules first.MIMIC is a very large dataset; hence we only selected some part of it. We included the tables from four modules: `core`, `ed`, `hosp`, and `icu`. We will take a glance at these modules first.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# Load the data 
# admissions = pd.read_csv('../data/core/admissions.csv')
patients = pd.read_csv('../data/core/patients.csv')
diagnose = pd.read_csv('../data/hosp/d_icd_diagnoses.csv')
subj_diagnose = pd.read_csv('../data/hosp/diagnoses_icd.csv')
vital_raw = pd.read_csv('../data/ed/vitalsign.csv')
icu_stays = pd.read_csv('../data/icu/icustays.csv')
item_names = pd.read_csv('../data/icu/d_items.csv').set_index('itemid')
chart_event = pd.read_csv('../data/icu/chart_event_filtered.csv')

## `core` module 

The core module stores patient tracking data for MIMIC-IV data analysis. The tables in this module list patient demographics, hospitalizations, and ward stays. 

The most important table is the `patients` table where the information that is consistent for the lifetime of a patient is stored. This will be used to identify each patient.

`dod` column (means "date of death") contains the information about mortality inside the hospital which will be used as target in this study.

In [3]:
patients.head(3)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000048,F,23,2126,2008 - 2010,NaN
1,10002723,F,0,2128,2017 - 2019,NaN
2,10003939,M,0,2184,2008 - 2010,NaN


**describe**

## `hosp` module

The hospital module contains electronic health record data during hospitalisation. Since we foucsed on ICU patients in this analysis, most of the tables in this module is not of our interest. The only information we need is the billed diagnosis of patients in order to select our cohort.

`diagnoses_icd.csv` contains the icd code for each subject and the corresponding icd version. With the code and version we can acquire the name of the diagnoses from the `d_icd_diagnoses.csv` file. 

In [4]:
subj_diagnose.head(3)

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,15734973,20475282,3,2825,9
1,15734973,20475282,2,V0251,9
2,15734973,20475282,5,V270,9


In [5]:
diagnose.head(3)

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"


## `ed` module

`ed` module contains emergency room data. Each edstays row has a unique `stay id` representing a patient stay in the ED. If the patient was hospitalised after their ED visit, `hadm` (hospital addmision) id will be populated. hadm id can be linked to MIMIC-IV to get hospital stay details.  `subject id` can be linked to MIMIC-IV for patient demographics. 

We only use `vitalsign` table in this module since vitalsigns like heart rate, respotary rate are directly related to death. And the first set of vital signs after patients admitted to emergency room is most important since they haven't received any medication. 

In [6]:
vital_raw.head(3)

,subject_id,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain
0,16113983,37539106,2116-06-10 00:32:00,98.2,82.0,15.0,NaN,106.0,72.0,NaN,NaN
1,15128994,30058281,2167-08-29 02:25:00,98.3,79.0,20.0,97.0,126.0,73.0,NaN,0
2,15128994,30058281,2167-08-29 04:51:00,97.6,79.0,20.0,98.0,126.0,73.0,NaN,0


## `icu` module

`icu` module contains the main data we are going to analyze. 

The `icu` data includes intravenous and fluid inputs, patient outputs, procedures, date/time information, and other charted information (chartevents). All events tables have a stay id column to identify ICU patients in icustays and an itemid column to identify concepts in d items.

`icustays.csv` tracks information for ICU stays including adminission and discharge times. This table will be used to identify the patients who were admitted to the icu since not all patients admitted to ICU. 

In [7]:
icu_stays.head(3)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,17867402,24528534,31793211,Trauma SICU (TSICU),Trauma SICU (TSICU),2154-03-03 04:11:00,2154-03-04 18:16:56,1.587454
1,14435996,28960964,31983544,Trauma SICU (TSICU),Trauma SICU (TSICU),2150-06-19 17:57:00,2150-06-22 18:33:54,3.025625
2,17609946,27385897,33183475,Trauma SICU (TSICU),Trauma SICU (TSICU),2138-02-05 18:54:00,2138-02-15 12:42:05,9.741725


`chart_event_filtered.csv` contains the majority of the data. During their ICU stay, the primary repository of a patient’s information is their electronic chart. The electronic chart displays patients’ routine vital signs and any additional information relevant to their care: ventilator settings, laboratory values, code status, mental status, and so on. Each event is represented by an `item_id` in this table and the real name of the event can be found in the `d_items.csv` file

`chart_event` table is fairly large, so we uploaded a filtered version. The code used for filtering were shown in appendix. 

In [8]:
chart_event.head(3)

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,Unnamed: 10
0,10006277,25610553,30888848,2176-06-08 01:18:00,2176-06-08 01:20:00,224876,Cloudy,0.0,NaN,NaN,NaN
1,10006277,25610553,30888848,2176-06-08 00:57:00,2176-06-08 01:00:00,223770,90,90.0,%,0.0,NaN
2,10006277,25610553,30888848,2176-06-08 04:18:00,2176-06-08 04:18:00,223988,Clear,0.0,NaN,NaN,NaN


In [9]:
item_names.head(3)

,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
itemid,,,,,,,,
220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN


# The Cohort 
Here we select the patients dignosed with ICH from the whole hospital addmission. The corresponding ICD codes are 'I61*' (ICD-10) and '431*' (ICD-9). 

In [10]:
diagnose[diagnose['icd_code'].str.contains('I61')].head(3)

,icd_code,icd_version,long_title
24846,I61,10,Nontraumatic intracerebral hemorrhage
24847,I610,10,Nontraumatic intracerebral hemorrhage in hemis...
24848,I611,10,Nontraumatic intracerebral hemorrhage in hemis...


In [11]:
diagnose[diagnose['icd_code'].str.contains('^431')].head(3)

,icd_code,icd_version,long_title
4850,431,9,Intracerebral hemorrhage


Then we filter the patients who were addmitteed to the intensive care unit. 

In [12]:
icu_ids = icu_stays['subject_id'].unique()

In [13]:
def get_icu_cohort(icd10, icd9):
    """
    Input parameters are the icd code for a certain disease. For higher reliability, they should be regular expression. 
    """
    # filter all diagnosed subjects
    df1=patients.set_index('subject_id') 
    df_icd10=subj_diagnose[subj_diagnose['icd_code'].str.contains(str(icd10))]
    df_icd9 = subj_diagnose[subj_diagnose['icd_code'].str.contains(str(icd9))]
    df_icdall=pd.concat([df_icd9,df_icd10]).drop_duplicates('subject_id', keep='first')
    # filter the patients admitted to icu
    df_icu = df_icdall[df_icdall['subject_id'].isin(icu_ids)].set_index('subject_id')
    data = df_icu.join(df1,how='left')
    # add the target, i.e., mortality flag to the table
    data['dod']=data['dod'].replace(np.nan, 0, regex=True)
    data['dod']=data['dod'].replace('-', 1, regex=True)

    return data

def mortality_rate(df):
    """Calculate the mortality rate for selected cohort. """
    return np.count_nonzero(df['dod']==1)/len(df['dod'])

In [14]:
cohort = get_icu_cohort('I61',r'^431')
print('cohort size', cohort.shape[0])
print('mortality rate', mortality_rate(cohort))
cohort

cohort size 2485
mortality rate 0.2784708249496982


,hadm_id,seq_num,icd_code,icd_version,gender,anchor_age,anchor_year,anchor_year_group,dod
subject_id,,,,,,,,,
12116269,24214849,2,431,9,M,66,2166,2008 - 2010,0
12155595,26672436,2,431,9,F,52,2168,2008 - 2010,0
19620109,26497452,2,431,9,F,66,2128,2008 - 2010,0
19330004,20802265,1,431,9,M,52,2187,2008 - 2010,0
18414729,22609366,2,431,9,F,60,2152,2008 - 2010,0
...,...,...,...,...,...,...,...,...,...
14050724,20952526,1,I615,10,F,57,2112,2011 - 2013,0
12557389,21294125,1,I613,10,F,79,2151,2017 - 2019,0
18065731,25556934,1,I618,10,M,46,2137,2011 - 2013,0


**Describe Cohort  Figures**

# Feature Engineering

In [15]:
def get_data(cohort_index, data_table, index_colname):
    return data_table[data_table[index_colname].isin(cohort_index)]

In [16]:
def fill_na_mean(df, rd=2, inplace=True):
    for col in list(df.columns[df.isnull().sum() > 0]):
        mean_val = df[col].mean()
        if inplace:
            try:
                df[col].fillna(round(mean_val,rd),inplace=inplace)
            except TypeError:
                continue
        else:
            try:
                df = df[col].fillna(round(mean_val,rd))
            except TypeError:
                continue
    if not inplace:
        return df

In [17]:
# cohort subject id is used to select data from other tables.
cohort_ind = cohort.index

## Vital Signs




In [18]:
vital_raw.head(3)

,subject_id,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain
0,16113983,37539106,2116-06-10 00:32:00,98.2,82.0,15.0,NaN,106.0,72.0,NaN,NaN
1,15128994,30058281,2167-08-29 02:25:00,98.3,79.0,20.0,97.0,126.0,73.0,NaN,0
2,15128994,30058281,2167-08-29 04:51:00,97.6,79.0,20.0,98.0,126.0,73.0,NaN,0


In [19]:
# Drop rhythm, pain which have a lot of missing data
vital_id = get_data(cohort_ind, vital_raw, 'subject_id' ).iloc[:,:-2]
vital_id.head()

,subject_id,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp
105,10046166,38848658,2132-12-06 11:53:00,97.0,75.0,12.0,NaN,154.0,73.0
261,10900387,30300312,2146-09-08 01:25:00,98.8,79.0,16.0,93.0,162.0,87.0
525,16652205,35474533,2169-04-09 18:55:00,NaN,73.0,27.0,98.0,169.0,79.0
526,16652205,35474533,2169-04-09 19:50:00,NaN,70.0,45.0,100.0,187.0,69.0
527,16652205,35474533,2169-04-09 21:00:00,98.4,70.0,30.0,98.0,138.0,58.0


In [20]:
fill_na_mean(vital_id)
vital_id

,subject_id,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp
105,10046166,38848658,2132-12-06 11:53:00,97.00,75.0,12.0,97.55,154.0,73.0
261,10900387,30300312,2146-09-08 01:25:00,98.80,79.0,16.0,93.00,162.0,87.0
525,16652205,35474533,2169-04-09 18:55:00,97.47,73.0,27.0,98.00,169.0,79.0
526,16652205,35474533,2169-04-09 19:50:00,97.47,70.0,45.0,100.00,187.0,69.0
527,16652205,35474533,2169-04-09 21:00:00,98.40,70.0,30.0,98.00,138.0,58.0
...,...,...,...,...,...,...,...,...,...
1650418,12557389,35529368,2153-01-24 23:15:00,97.47,75.0,23.0,98.00,119.0,62.0
1650902,14745196,36076575,2204-06-17 12:14:00,97.20,89.0,16.0,98.00,129.0,86.0
1650903,14745196,36076575,2204-06-17 14:59:00,98.90,88.0,18.0,97.00,166.0,91.0
1650904,14745196,36076575,2204-06-17 18:33:00,99.50,95.0,20.0,94.00,153.0,67.0


In [21]:
vital_id =vital_id.drop_duplicates('subject_id', keep='last').set_index('subject_id')

In [22]:
vital_id.head()

,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp
subject_id,,,,,,,,
16652205,31288294,2169-04-22 23:50:00,101.40,100.0,44.0,97.55,97.0,61.0
11213607,32187974,2194-06-11 01:48:00,98.40,98.0,20.0,98.00,152.0,80.0
19620109,36855170,2132-10-16 23:43:00,98.50,67.0,15.0,100.00,101.0,67.0
16379037,37641494,2177-04-16 23:54:00,98.00,100.0,19.0,98.00,145.0,80.0
15936063,38505545,2161-05-22 16:30:00,97.47,118.0,20.0,98.00,145.0,72.0


In [29]:
final = cohort.join(vital_id)
final.shape

(2485, 17)

## Chartted Event

In [23]:
chart_event.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,Unnamed: 10
0,10006277,25610553,30888848,2176-06-08 01:18:00,2176-06-08 01:20:00,224876,Cloudy,0.0,NaN,NaN,NaN
1,10006277,25610553,30888848,2176-06-08 00:57:00,2176-06-08 01:00:00,223770,90,90.0,%,0.0,NaN
2,10006277,25610553,30888848,2176-06-08 04:18:00,2176-06-08 04:18:00,223988,Clear,0.0,NaN,NaN,NaN
3,10006277,25610553,30888848,2176-06-08 01:16:00,2176-06-08 01:16:00,220739,Spontaneously,4.0,0,NaN,NaN
4,10006277,25610553,30888848,2176-06-08 11:00:00,2176-06-08 12:20:00,224650,None,0.0,NaN,NaN,NaN


In [24]:
chart_pivot= chart_event.pivot(index='subject_id', columns='itemid', values='value')
chart_pivot.head()

itemid,220045,220046,220047,220048,220179,220180,220181,220210,220228,220277,...,227443,227457,227465,227466,227467,227944,227968,227969,228096,228299
subject_id,,,,,,,,,,,,,,,,,,,,,
10006277,95,120,60,AF (Atrial Fibrillation),139,82,94,37,13,95,...,25,178,18.5,28.1,2,3 rails up,Yes,Bed locked in low position,NaN,NaN
10007677,84,130,50,SR (Sinus Rhythm),88,51,58,18,9.8,95,...,26,193,13.8,27.4,1.3,3 rails up,Yes,Pain evaluated and treated,0 Alert and calm,0 Alert and calm
10013310,100,130,50,ST (Sinus Tachycardia),116,61,84,19,8.8,100,...,27,290,12.8,27.6,1.3,3 rails up,Yes,Lines and tubes concealed,0 Alert and calm,0 Alert and calm
10017492,88,120,60,1st AV (First degree AV Block),149,63,71,25,8.4,98,...,23,357,11.1,23.7,1,3 rails up,Yes,Adequate lighting,0 Alert and calm,0 Alert and calm
10025463,81,120,60,ST (Sinus Tachycardia),122,81,91,22,NaN,97,...,NaN,NaN,NaN,NaN,NaN,3 rails up,Yes,Pain evaluated and treated,"-5 Unarousable, no response to voice or physic...","-2 Light sedation, briefly awakens to voice (e..."


In [25]:
item_dict = dict()
for item_id in chart_pivot.columns:
    item_dict[item_id] = item_names.loc[item_id, 'abbreviation']

In [26]:
feature_list = np.array(list(item_dict.values()))
chart_pivot.columns=feature_list
feature_list[:5]

array(['HR', 'HR Alarm - High', 'HR Alarm - Low', 'Heart Rhythm', 'NBPs'],
      dtype='<U43')

In [27]:
chart_pivot

,HR,HR Alarm - High,HR Alarm - Low,Heart Rhythm,NBPs,NBPd,NBPm,RR,Hemoglobin,SpO2,...,HCO3 (serum),Platelet Count,PT,PTT,INR,Side Rails,All Medications Tolerated,Safety Measures,Richmond-RAS Scale,Goal Richmond-RAS Scale
subject_id,,,,,,,,,,,,,,,,,,,,,
10006277,95,120,60,AF (Atrial Fibrillation),139,82,94,37,13,95,...,25,178,18.5,28.1,2,3 rails up,Yes,Bed locked in low position,NaN,NaN
10007677,84,130,50,SR (Sinus Rhythm),88,51,58,18,9.8,95,...,26,193,13.8,27.4,1.3,3 rails up,Yes,Pain evaluated and treated,0 Alert and calm,0 Alert and calm
10013310,100,130,50,ST (Sinus Tachycardia),116,61,84,19,8.8,100,...,27,290,12.8,27.6,1.3,3 rails up,Yes,Lines and tubes concealed,0 Alert and calm,0 Alert and calm
10017492,88,120,60,1st AV (First degree AV Block),149,63,71,25,8.4,98,...,23,357,11.1,23.7,1,3 rails up,Yes,Adequate lighting,0 Alert and calm,0 Alert and calm
10025463,81,120,60,ST (Sinus Tachycardia),122,81,91,22,NaN,97,...,NaN,NaN,NaN,NaN,NaN,3 rails up,Yes,Pain evaluated and treated,"-5 Unarousable, no response to voice or physic...","-2 Light sedation, briefly awakens to voice (e..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19992425,63,120,40,SB (Sinus Bradycardia),91,54,62,9,13.2,96,...,27,129,13.5,26.9,1.3,3 rails up,Yes,Call light within reach,0 Alert and calm,0 Alert and calm
19992885,73,130,50,ST (Sinus Tachycardia),133,52,80,13,7.7,100,...,28,133,17.4,39.4,1.6,3 rails up,Yes,Bed locked in low position,"-2 Light sedation, briefly awakens to voice (e...",0 Alert and calm
19994233,62,120,45,AF (Atrial Fibrillation),114,51,71,17,11.5,97,...,30,220,11.8,27.4,1.1,3 rails up,NaN,Pain evaluated and treated,0 Alert and calm,0 Alert and calm


In [28]:
final = final.join(chart_pivot)

NameError: name 'final' is not defined

In [35]:
test = final.iloc[:,:-5].dropna(how='any',axis=0)
test[['dod']] = test[['dod']].astype('int')
test.describe()

,hadm_id,seq_num,icd_version,anchor_age,anchor_year,dod,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp
count,6.080000e+02,608.000000,608.000000,608.000000,608.000000,608.000000,6.080000e+02,608.000000,608.000000,608.000000,608.000000,608.000000,608.000000
mean,2.493169e+07,3.166118,9.523026,64.560855,2152.975329,0.210526,3.501578e+07,97.689984,81.041941,18.415132,97.604112,129.319079,70.958882
std,2.855539e+06,4.732763,0.499881,15.039199,23.288345,0.408018,2.941544e+06,4.403925,16.703869,4.120856,4.421384,20.274210,14.681154
min,2.003789e+07,1.000000,9.000000,19.000000,2110.000000,0.000000,3.001288e+07,34.700000,42.000000,2.000000,2.000000,10.000000,23.000000
25%,2.257716e+07,1.000000,9.000000,55.000000,2134.000000,0.000000,3.255622e+07,97.470000,70.000000,16.000000,96.000000,115.000000,61.000000
50%,2.492227e+07,1.000000,10.000000,66.000000,2152.000000,0.000000,3.498434e+07,97.800000,80.000000,18.000000,98.000000,129.000000,71.000000
75%,2.738553e+07,3.000000,10.000000,76.000000,2173.000000,0.000000,3.766286e+07,98.300000,92.000000,20.000000,100.000000,140.250000,79.000000
max,2.999962e+07,38.000000,10.000000,91.000000,2202.000000,1.000000,3.996534e+07,105.000000,138.000000,44.000000,100.000000,196.000000,130.000000


In [37]:
test.shape

(608, 143)

In [ ]:
item_names.head()

# Description

Index(['charttime', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp',
       'dbp', 'HR', 'HR Alarm - High', 'HR Alarm - Low',
       ...
       'IV/Saline lock', 'Gait/Transferring', 'Mental status',
       '20 Gauge Dressing Occlusive', 'Potassium (serum)', 'HCO3 (serum)',
       'Platelet Count', 'PT', 'PTT', 'INR'],
      dtype='object', length=133)

# Modelling

In [39]:
from DS_MIMIC_knn import train_test_split

In [40]:
test[test.dod==0].shape[0]/test.shape[0]

0.7894736842105263

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

X = test[test.columns[11:]]
y = test['dod']

X_train_orig, X_test_orig, y_train, y_test = train_test_split(X, y,0.2)
std= StandardScaler()
X_train=std.fit_transform(X_train_orig)
X_test=std.fit_transform(X_test_orig)

ValueError: could not convert string to float: 'SR (Sinus Rhythm)'

In [ ]:
X_train.shape

## Logistic Regression

In [ ]:
# check classification scores of logistic regression
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
print('Train/Test split results:')
print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

In [ ]:
idx = np.min(np.where(tpr > 0.95)) # index of the first threshold for which the sensibility > 0.95

plt.figure()
plt.plot(fpr, tpr, color='coral', label='ROC curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot([0,fpr[idx]], [tpr[idx],tpr[idx]], 'k--', color='blue')
plt.plot([fpr[idx],fpr[idx]], [0,tpr[idx]], 'k--', color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

print("Using a threshold of %.3f " % thr[idx] + "guarantees a sensitivity of %.3f " % tpr[idx] +  
      "and a specificity of %.3f" % (1-fpr[idx]) + 
      ", i.e. a false positive rate of %.2f%%." % (np.array(fpr[idx])*100))

## Random Forest

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Fit the model
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train_orig, y_train)

# Use score method to calculate the accuracy over the whole test set
acc=rf.score(X_test_orig, y_test)
print(acc)
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X, y)


ft_imp = pd.Series(rf.feature_importances_).sort_values(ascending=False)
ft_imp.head(10)
ft_imp

In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(rf)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(X_test_orig)

# Make plot. Index of [1] is explained in text below.
shap.summary_plot(shap_values[1], X_test_orig)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(40)
test_scores = []
train_scores = []

for i in range(1,15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))


In [ ]:
max_train_score = max(train_scores)
train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
print('Max train score {} % and k = {}'.format(max_train_score*100,list(map(lambda x: x+1, train_scores_ind))))

In [ ]:
## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

In [ ]:
plt.figure(figsize=(12,5))
p = sns.lineplot(x=range(1,15),y=train_scores,marker='*',label='Train Score')
p = sns.lineplot(x=range(1,15),y=test_scores,marker='o',label='Test Score')

## My KNN


def My_knncomper_difftest( X_train, X_test, y_train, y_test,kmax):
    rightnumber = 0           
    print('enter phase 2')
    Train_acc=[]
    Test_acc=[]
    print(" maximum K value is "+str(kmax))           
    for k in range(1,kmax):
        test_predict_score=0
        rightnumber=0
        predictlist =MYknn(X_test, X_train, y_train, k)
        for m in range(len(X_test)):
            if predictlist[m] == y_test[m]:
                rightnumber = rightnumber + 1
        test_predict_score = (rightnumber / len(y_test))
        print(test_predict_score*100)
        Test_acc.append(  test_predict_score * 100)
        rightnumber=0
        Train_predict_score=0
        predictlist = MYknn(X_train, X_train, y_train, k)
        for m in range(len(X_train)):
            if predictlist[m] == y_train[m]:
                rightnumber = rightnumber + 1
        Train_predict_score = (rightnumber / len(X_train))
        Train_acc.append( Train_predict_score * 100)      

    plt.plot(Test_acc)
    plt.title('Test_acc')
    plt.show()
    plt.plot(Train_acc)
    plt.title('Train_acc')
    plt.show()

   


def MYknn(test_object, training_object, training_object_target, K):
    predictlist = []
    for newpoint in test_object:
        dataSetSize = training_object.shape[0]
        diffMat= np.tile(newpoint, (dataSetSize, 1)) - training_object
        sqDistances = (diffMat**2).sum(axis=1)
        distances = sqDistances ** 0.5
        sortedDistIndicies = distances.argsort()
        classCount = {}
        for i in range(K):
            voteIlabel = training_object_target[sortedDistIndicies[i]]
            classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1  #Get the value of key from the map and return 0 without key
            sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
        predictlist.append(sortedClassCount[0][0])
    return predictlist

My_knncomper_difftest(X_train, X_test, y_train, y_test, 15)

In [30]:
def My_knncomper_difftest(kmax):
    rightnumber = 0           
    print('enter phase 2')
    Train_acc=[]
    Test_acc=[]  
    print(" maximum K value is "+str(kmax))           
    for k in range(1,kmax):
        test_predict_score=0
        rightnumber=0
        predictlist =MYknn(X_test_std, X_train_std, Y_train, k)
        for m in range(len(X_test_std)):
            if predictlist[m] == Y_test[m]:
                rightnumber = rightnumber + 1
        test_predict_score = (rightnumber / len(Y_test))
#         print(test_predict_score*100)
        Test_acc.append(  test_predict_score * 100)
        rightnumber=0
        Train_predict_score=0
        predictlist = MYknn(X_train_std, X_train_std, Y_train, k)
        for m in range(len(X_train_std)):
            if predictlist[m] == Y_train[m]:
                rightnumber = rightnumber + 1
        Train_predict_score = (rightnumber / len(X_train_std))
        Train_acc.append( Train_predict_score * 100)      
     
    plt.plot(Test_acc)
    plt.title('Test_acc')
    plt.show()
    plt.plot(Train_acc)
    plt.title('Train_acc')
    plt.show()

        
def MYknn(test_object, training_object, training_object_target, K):
    predictlist = []
    for newpoint in test_object:
        dataSetSize = training_object.shape[0]
        # caculate the distance matrix between new point and all training  points 
        diffMat= np.tile(newpoint, (dataSetSize, 1)) - training_object
        sqDistances = (diffMat**2).sum(axis=1)
        distances = sqDistances ** 0.5
        # argsort: sort array and return array index,keep the orignal array.
        sortedDistIndicies = distances.argsort()
        classCount = {}
        
        for i in range(K):
            # get lables form 0 to k nearest training points
            voteIlabel = training_object_target[sortedDistIndicies[i]]
            #Count different lables number 
            classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1  
            # sort lable and get the most lable
            sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
        # store predict newpoint label 
        predictlist.append(sortedClassCount[0][0])
    return predictlist

def train_test_split(X, y,train_size):
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, train_size)
    X_train = X[split]
    y_train = y[split]
    X_test =  X[~split]
    y_test =  y[~split]
    return X_train, X_test,y_train, y_test


In [50]:
MIMIC_Data=final.drop(['hadm_id','seq_num','icd_code','icd_version','gender', 'anchor_year', 'anchor_year_group' ,'stay_id','charttime'],axis=1)
MIMIC_Data

,anchor_age,dod,temperature,heartrate,resprate,o2sat,sbp,dbp
subject_id,,,,,,,,
12116269,66,0,NaN,NaN,NaN,NaN,NaN,NaN
12155595,52,0,NaN,NaN,NaN,NaN,NaN,NaN
19620109,66,0,98.50,67.0,15.0,100.00,101.0,67.0
19330004,52,0,97.60,66.0,16.0,100.00,166.0,88.0
18414729,60,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
14050724,57,0,98.30,69.0,17.0,100.00,148.0,76.0
12557389,79,0,97.47,75.0,23.0,98.00,119.0,62.0
18065731,46,0,97.47,87.0,18.0,97.55,126.0,75.0


In [51]:
MIMIC_Data['dod'] = MIMIC_Data['dod'].astype(np.int64)
fill_na_mean(MIMIC_Data)

In [52]:
MIMIC_Data

,anchor_age,dod,temperature,heartrate,resprate,o2sat,sbp,dbp
subject_id,,,,,,,,
12116269,66,0,97.31,79.83,18.16,97.58,130.49,74.04
12155595,52,0,97.31,79.83,18.16,97.58,130.49,74.04
19620109,66,0,98.50,67.00,15.00,100.00,101.00,67.00
19330004,52,0,97.60,66.00,16.00,100.00,166.00,88.00
18414729,60,0,97.31,79.83,18.16,97.58,130.49,74.04
...,...,...,...,...,...,...,...,...
14050724,57,0,98.30,69.00,17.00,100.00,148.00,76.00
12557389,79,0,97.47,75.00,23.00,98.00,119.00,62.00
18065731,46,0,97.47,87.00,18.00,97.55,126.00,75.00


In [49]:
MIMIC_Data.columns

Index(['anchor_age', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp',
       'dbp'],
      dtype='object')

In [46]:
import numpy as np
import pandas as pd
import operator
import matplotlib.pyplot as plt

In [47]:

np.random.seed(40)
X_train, X_test, Y_train, Y_test= train_test_split(MIMIC_X, MIMIC_Y,train_size=70)
X_train_std=(X_train-X_train.mean())/X_train.std()
X_test_std=(X_test-X_test.mean())/X_test.std()
print('use my knn')

My_knncomper_difftest(20)

KeyError: 'dod'

# Appendix

We select chart event data from Google Bigquery using SQL. This command took about 20 mins to run, and returns milions of rows of data.
So won't run it in the notebook, we filtered the data and saved it locally. 

In [ ]:
for i in range(5):
    sql = f"""SELECT * 
    FROM `physionet-data.mimic_icu.chartevents`
    WHERE subject_id in {tuple(cohort.index[497*i:497*(i+1)].values.tolist())}
    ORDER BY subject_id"""

    df = pd.read_gbq(sql, project_id='focus-dragon-313813', dialect='standard', use_bqstorage_api=True)
    df.to_csv(f'df{i}.csv')